In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 3, Finished, Available, Finished)

In [2]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b2e2dffb-0bf3-4622-bce8-881a25e4a900)

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 5, Finished, Available, Finished)

In [4]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error")
)

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 6, Finished, Available, Finished)

In [5]:
result = model.transform(df)

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 7, Finished, Available, Finished)

In [6]:
display(result)

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6166b380-fac5-4aec-8fe7-c8efbe3534df)

In [7]:
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment",col("response.documents.sentiment"))

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 9, Finished, Available, Finished)

In [8]:
display(sentiment_df)

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 539cf5e9-69a2-484f-86ee-4d7bf7b2b182)

In [9]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 11, Finished, Available, Finished)

In [10]:
display(sentiment_df_final)

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ec6fcb82-4535-494c-8285-e1df74c79129)

In [12]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_anlysis'

    sentiment_df_final.write.format("delta").saveAsTable("bing_lake_db.tbl_sentiment_anlysis")

except AnalysisException:

    print("Table alredy Exists")

    sentiment_df_final.createOrReplaceTempView("vw_df_sentiment_final")

    spark.sql(f"""
                MERGE INTO {table_name} target_table
                USING vw_df_sentiment_final source_view
                ON source_view.url = target_table.url
                WHEN MATCHED AND 
                    (source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished OR
                    source_view.sentiment <> target_table.sentiment)
                THEN 
                    UPDATE SET 
                    target_table.title = source_view.title,
                    target_table.description = source_view.description,
                    target_table.category = source_view.category,
                    target_table.image = source_view.image,
                    target_table.provider = source_view.provider,
                    target_table.datePublished = source_view.datePublished,
                    target_table.sentiment = source_view.sentiment

                WHEN NOT MATCHED THEN 
                    INSERT (title, description, category, url, image, provider, datePublished,sentiment)
                    VALUES (source_view.title, source_view.description, source_view.category, 
                            source_view.url, source_view.image, source_view.provider, 
                            source_view.datePublished,source_view.sentiment)
    """)

StatementMeta(, 15e98ab4-18ac-42ff-bf1d-761fee7c90c4, 14, Finished, Available, Finished)

Table alredy Exists
